In [211]:
opt = list()
opt$sumstats = "/home/hs3163/Project/one_gene/data/WAIST.sumstat"
opt$weights = "/home/hs3163/Project/one_gene/Whole_Blood/WEIGHTS/GTEx_V8.weights_list.txt"
opt$weights_dir = "/"
opt$ref_ld_chr = "/home/hs3163/Project/one_gene/data/GTEx.ref" 
opt$chr =  7 
opt$max_impute = 0.5 
opt$out = "/home/hs3163/Project/one_gene/WAIST.sumstat/Whole_Blood/result/WAIST_chr7.twas.txt"


In [212]:
library(dplyr)
library(tibble)
library(readr)

In [213]:
sumstat = read_delim("/home/hs3163/Project/one_gene/data/WAIST.sumstat",delim = "\t")
sumstat = sumstat%>%mutate(A1 = REF,A2 = ALT)
sumstat%>%write_delim("/home/hs3163/Project/one_gene/data/WAIST.sumstat",delim = "\t")


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  CHR = col_double(),
  POS = col_double(),
  REF = col_character(),
  ALT = col_character(),
  SNP = col_character(),
  BETA = col_double(),
  SE = col_double(),
  P = col_double(),
  cood38 = col_double(),
  cood37 = col_double(),
  qc = col_double(),
  Dif = col_double(),
  SNP_new = col_character(),
  A1 = col_character(),
  A2 = col_character()
)




In [214]:
allele.qc = function(a1,a2,ref1,ref2) {
        a1 = toupper(a1)
        a2 = toupper(a2)
        ref1 = toupper(ref1)
        ref2 = toupper(ref2)

	ref = ref1
	flip = ref
	flip[ref == "A"] = "T"
	flip[ref == "T"] = "A"
	flip[ref == "G"] = "C"
	flip[ref == "C"] = "G"
	flip1 = flip

	ref = ref2
	flip = ref
	flip[ref == "A"] = "T"
	flip[ref == "T"] = "A"
	flip[ref == "G"] = "C"
	flip[ref == "C"] = "G"
	flip2 = flip;

	snp = list()
	snp[["keep"]] = !((a1=="A" & a2=="T") | (a1=="T" & a2=="A") | (a1=="C" & a2=="G") | (a1=="G" & a2=="C"))
	snp[["keep"]][ a1 != "A" & a1 != "T" & a1 != "G" & a1 != "C" ] = F
	snp[["keep"]][ a2 != "A" & a2 != "T" & a2 != "G" & a2 != "C" ] = F
	snp[["flip"]] = (a1 == ref2 & a2 == ref1) | (a1 == flip2 & a2 == flip1)

	return(snp)
}

In [215]:
sumstat = read.table(opt$sumstats,head=T,as.is=T)
wgtlist = read.table(opt$weights,head=T,as.is=T)
wgtlist = wgtlist[ as.character(wgtlist$CHR) == as.character(opt$chr) , ]
chr = unique(wgtlist$CHR)
N = nrow(wgtlist)
out.tbl = data.frame( "PANEL" = rep(NA,N) , "FILE" = character(N) , "ID" = character(N) , "CHR" = numeric(N) , "P0" = character(N) , "P1" = character(N) ,"HSQ" = numeric(N) , "BEST.GWAS.ID" = character(N) , "BEST.GWAS.Z" = numeric(N) , "EQTL.ID" = character(N) , "EQTL.R2" = numeric(N) , "EQTL.Z" = numeric(N) , "EQTL.GWAS.Z" = numeric(N) , "NSNP" = numeric(N) , "NWGT" = numeric(N) , "MODEL" = character(N) , "MODELCV.R2" = character(N) , "MODELCV.PV" = character(N) , "TWAS.Z" = numeric(N) , "TWAS.P" = numeric(N) , stringsAsFactors=FALSE )

Warning message in read.table(opt$weights, head = T, as.is = T):
“incomplete final line found by readTableHeader on '/home/hs3163/Project/one_gene/Whole_Blood/WEIGHTS/GTEx_V8.weights_list.txt'”


In [216]:
genos = list()
genos$bim = read.table("/home/hs3163/Project/one_gene/data/GTEx.ref.bim")

In [217]:
genos$bim = genos$bim%>%filter(V4 > sumstat$cood38[1],V4 < sumstat$cood38[nrow(sumstat)])

In [218]:
sumstat = sumstat%>%mutate(A1 = REF,A2 = ALT)

In [219]:
m = match( genos$bim[,2] , sumstat$SNP )
sum.missing = is.na(m)

In [220]:
sumstat = sumstat[m,]
sumstat$SNP = genos$bim[,2]

In [221]:
sumstat$A1[ sum.missing ] = as.character(genos$bim[sum.missing,5])
sumstat$A2[ sum.missing ] = as.character(genos$bim[sum.missing,6])

In [223]:
qc = allele.qc( sumstat$A1 , sumstat$A2 , genos$bim[,5] , genos$bim[,6] )

In [224]:
sumstat$Z[ qc$flip ] = -1 * sumstat$Z[ qc$flip ]

In [225]:
sumstat$A1[ qc$flip ] = genos$bim[qc$flip,5]
sumstat$A2[ qc$flip ] = genos$bim[qc$flip,6]

In [226]:
if ( sum(!qc$keep) > 0 ) {
	genos$bim = genos$bim[qc$keep,]
	genos$bed = genos$bed[,qc$keep]
	sumstat = sumstat[qc$keep,]
}

In [227]:
FAIL.ctr = 0
w = 1


In [228]:
wgt.file = paste(opt$weights_dir,"/",wgtlist$WGT[w],sep='')

In [229]:
wgt.file

[1] "///home/hs3163/Project/one_gene/Whole_Blood/WEIGHTS/GTEx_V8.ENSG00000153814.wgt.RDat"

In [230]:
	load(wgt.file)



In [202]:
	# Remove NAs (these should not be here)
	wgt.matrix[is.na(wgt.matrix)] = 0
m = match( snps[,2] , genos$bim[,2] )
	m.keep = !is.na(m)

In [231]:
snps = snps[m.keep,]

In [232]:
cur.bim = genos$bim[m[m.keep],]

In [233]:
cur.bim

,V1,V2,V3,V4,V5,V6
,<int>,<fct>,<int>,<int>,<fct>,<fct>
NA,NA,NA,NA,NA,NA,NA
NA.1,NA,NA,NA,NA,NA,NA
NA.2,NA,NA,NA,NA,NA,NA
NA.3,NA,NA,NA,NA,NA,NA
NA.4,NA,NA,NA,NA,NA,NA
NA.5,NA,NA,NA,NA,NA,NA
4,7,chr7_27181165_C_A_b38,0,27181165,C,A
8,7,chr7_27181189_C_T_b38,0,27181189,C,T
9,7,chr7_27181212_A_G_b38,0,27181212,A,G


In [234]:

	# Match up the SNPs and weights
	snps = snps[m.keep,]
	wgt.matrix = wgt.matrix[m.keep,,drop=F]
	cur.genos = scale(genos$bed[,m[m.keep]])
	cur.bim = genos$bim[m[m.keep],]

ERROR: Error in array(x, c(length(x), 1L), if (!is.null(names(x))) list(names(x), : 'data' must be of a vector type, was 'NULL'


In [ ]:
a1 = toupper(a1)
a2 = toupper(a2)
ref1 = toupper(ref1)
ref2 = toupper(ref2)

	ref = ref1
	flip = ref
	flip[ref == "A"] = "T"
	flip[ref == "T"] = "A"
	flip[ref == "G"] = "C"
	flip[ref == "C"] = "G"
	flip1 = flip

	ref = ref2
	flip = ref
	flip[ref == "A"] = "T"
	flip[ref == "T"] = "A"
	flip[ref == "G"] = "C"
	flip[ref == "C"] = "G"
	flip2 = flip;

	snp = list()
	snp[["keep"]] = !((a1=="A" & a2=="T") | (a1=="T" & a2=="A") | (a1=="C" & a2=="G") | (a1=="G" & a2=="C"))
	snp[["keep"]][ a1 != "A" & a1 != "T" & a1 != "G" & a1 != "C" ] = F
	snp[["keep"]][ a2 != "A" & a2 != "T" & a2 != "G" & a2 != "C" ] = F
	snp[["flip"]] = (a1 == ref2 & a2 == ref1) | (a1 == flip2 & a2 == flip1)

